# MArch MAdness Models, 2024

Data From Kaggle Competition Files

Year two of trying this. Going to use the actual evaluation Functions intead of just beating my frined's picks

### PAckages

In [1]:
import os
import pandas as pd
import numpy as np
import sklearn
import random

In [2]:
os.chdir('C:\\Users\\Nick\\Python Projects\\2024 March Madness\\MMML')

### Data Import

In [3]:
# I want Mteams, MRegularSeasonDetailedResults, MTeamConferences, MTourneyCompactResults, MTeamConferences
# MMasseyOrdinals, MTeamSpellings, 2024_tourney_seeds (not updated id bet)

MTeams = pd.read_csv('Data\\MTeams.csv')
WTeams = pd.read_csv('Data\\WTeams.csv')

MRegSeason = pd.read_csv('Data\\MRegularSeasonDetailedResults.csv')
WRegSeason = pd.read_csv('Data\\WRegularSeasonDetailedResults.csv')

MConf = pd.read_csv('Data\\MTeamConferences.csv')
WConf = pd.read_csv('Data\\WTeamConferences.csv')

MTourneyResults = pd.read_csv('Data\\MNCAATourneyCompactResults.csv')
WTourneyResults = pd.read_csv('Data\\WNCAATourneyCompactResults.csv')

#MMassey = pd.read_csv('Data\\MMasseyOrdinals.csv')

MConfTOurney = pd.read_csv('Data\\MConferenceTourneyGames.csv')


In [4]:
RegSeason = pd.concat([MRegSeason, WRegSeason], ignore_index = True)
RegSeason.head(10)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
5,2003,11,1458,81,1186,55,H,0,26,57,...,11,12,17,6,22,8,19,4,3,25
6,2003,12,1161,80,1236,62,H,0,23,55,...,15,20,28,9,21,11,30,10,4,28
7,2003,12,1186,75,1457,61,N,0,28,62,...,17,17,23,8,25,10,15,14,8,18
8,2003,12,1194,71,1156,66,N,0,28,58,...,18,12,27,13,26,13,25,8,2,18
9,2003,12,1458,84,1296,56,H,0,32,67,...,14,7,12,9,23,10,18,1,3,18


In [5]:

SeasonWins = RegSeason.groupby(['WTeamID', 'Season'])['LTeamID'].count().reset_index().rename(columns = {'LTeamID':'Wins', 'WTeamID':'TeamID'})
SeasonLs = RegSeason.groupby(['LTeamID', 'Season'])['WTeamID'].count().reset_index().rename(columns = {'WTeamID':'Losses', 'LTeamID':'TeamID'})
OpponentsW = RegSeason.groupby(['WTeamID', 'Season'])['LTeamID'].apply(list).reset_index().rename(columns = {'LTeamID':'OpponentsBeaten', 'WTeamID':'TeamID'})
OpponentsL = RegSeason.groupby(['LTeamID', 'Season'])['WTeamID'].apply(list).reset_index().rename(columns = {'WTeamID':'OpponentsLost', 'LTeamID':'TeamID'})

#Merge these Columns
merged_df = pd.merge(SeasonWins, SeasonLs, on=['TeamID', 'Season'], how='outer')

merged_df1 = pd.merge(merged_df, OpponentsW, on=['TeamID', 'Season'], how='left')

TeamRecords = pd.merge(merged_df1, OpponentsL, on=['TeamID', 'Season'], how='left')


TeamRecords

,TeamID,Season,Wins,Losses,OpponentsBeaten,OpponentsLost
0,1101,2014,2.0,19.0,"[1146, 1223]","[1182, 1382, 1268, 1234, 1395, 1462, 1406, 130..."
1,1101,2015,7.0,21.0,"[1170, 1354, 1115, 1146, 1311, 1322, 1223]","[1409, 1182, 1415, 1222, 1260, 1129, 1213, 137..."
2,1101,2016,9.0,18.0,"[1197, 1146, 1322, 1270, 1249, 1223, 1249, 136...","[1339, 1161, 1324, 1306, 1304, 1415, 1395, 117..."
3,1101,2017,9.0,16.0,"[1306, 1149, 1372, 1249, 1309, 1358, 1322, 127...","[1149, 1328, 1307, 1170, 1362, 1309, 1368, 114..."
4,1101,2018,12.0,15.0,"[1144, 1132, 1102, 1402, 1358, 1270, 1223, 132...","[1117, 1252, 1367, 1252, 1403, 1309, 1368, 137..."
...,...,...,...,...,...,...
12852,3421,2013,NaN,28.0,NaN,"[3441, 3459, 3184, 3314, 3378, 3144, 3367, 324..."
12853,3438,2021,NaN,5.0,NaN,"[3416, 3187, 3241, 3155, 3199]"
12854,3451,2013,NaN,29.0,NaN,"[3140, 3129, 3329, 3119, 3428, 3307, 3430, 346..."
12855,3464,2010,NaN,30.0,NaN,"[3189, 3219, 3245, 3145, 3419, 3451, 3338, 310..."


Deal With the winless and undefeated seasons

In [6]:
TeamRecords['Wins'].fillna(0, inplace = True)
TeamRecords['Losses'].fillna(0, inplace = True)

In [7]:
TeamRecords['Win_PCT'] = TeamRecords['Wins']/(TeamRecords['Wins'] + TeamRecords['Losses'])
#TeamRecords

In [8]:
# Define a function to fill NaN values with an empty list
def fill_na_with_empty_list(value):
    if isinstance(value, float) and np.isnan(value):
        return []
    else:
        return value

# Fill NaN values with an empty list using applymap() and the defined function
TeamRecords = TeamRecords.applymap(fill_na_with_empty_list)

In [9]:
TeamRecords.isna().sum()

TeamID             0
Season             0
Wins               0
Losses             0
OpponentsBeaten    0
OpponentsLost      0
Win_PCT            0
dtype: int64

In [10]:
TeamRecords

,TeamID,Season,Wins,Losses,OpponentsBeaten,OpponentsLost,Win_PCT
0,1101,2014,2.0,19.0,"[1146, 1223]","[1182, 1382, 1268, 1234, 1395, 1462, 1406, 130...",0.095238
1,1101,2015,7.0,21.0,"[1170, 1354, 1115, 1146, 1311, 1322, 1223]","[1409, 1182, 1415, 1222, 1260, 1129, 1213, 137...",0.250000
2,1101,2016,9.0,18.0,"[1197, 1146, 1322, 1270, 1249, 1223, 1249, 136...","[1339, 1161, 1324, 1306, 1304, 1415, 1395, 117...",0.333333
3,1101,2017,9.0,16.0,"[1306, 1149, 1372, 1249, 1309, 1358, 1322, 127...","[1149, 1328, 1307, 1170, 1362, 1309, 1368, 114...",0.360000
4,1101,2018,12.0,15.0,"[1144, 1132, 1102, 1402, 1358, 1270, 1223, 132...","[1117, 1252, 1367, 1252, 1403, 1309, 1368, 137...",0.444444
...,...,...,...,...,...,...,...
12852,3421,2013,0.0,28.0,[],"[3441, 3459, 3184, 3314, 3378, 3144, 3367, 324...",0.000000
12853,3438,2021,0.0,5.0,[],"[3416, 3187, 3241, 3155, 3199]",0.000000
12854,3451,2013,0.0,29.0,[],"[3140, 3129, 3329, 3119, 3428, 3307, 3430, 346...",0.000000
12855,3464,2010,0.0,30.0,[],"[3189, 3219, 3245, 3145, 3419, 3451, 3338, 310...",0.000000


Season stats need to be by team so I cant just group by Wteam because then I'd only have their stats from games they won. 
I also can't have Wteam columns in the game and tourney games data when I  train/make predictions. I need to somehow randomize this so sometimes Wteam is T1 and sometimes T2

In [11]:
RegSeason.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [12]:
RegSeasonSwitch = RegSeason[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore',  'WLoc',
       'NumOT', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]
RegSeasonSwitch

,Season,DayNum,LTeamID,LScore,WTeamID,WScore,WLoc,NumOT,LFGM,LFGA,...,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF
0,2003,10,1328,62,1104,68,N,0,22,53,...,14,11,18,14,24,13,23,7,1,22
1,2003,10,1393,63,1272,70,N,0,24,67,...,20,10,19,15,28,16,13,4,4,18
2,2003,11,1437,61,1266,73,N,0,22,73,...,18,17,29,17,26,15,10,5,2,25
3,2003,11,1457,50,1296,56,N,0,18,49,...,9,17,31,6,19,11,12,14,2,18
4,2003,11,1208,71,1400,77,N,0,24,62,...,14,11,13,17,22,12,14,4,4,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186657,2024,100,3427,70,3409,74,H,0,26,65,...,25,13,18,5,22,19,19,8,4,20
186658,2024,100,3161,64,3424,67,H,0,26,59,...,27,6,13,11,28,14,11,4,0,12
186659,2024,100,3296,47,3444,65,A,0,17,50,...,22,17,27,4,29,11,7,11,1,16
186660,2024,100,3347,51,3457,54,H,1,22,61,...,18,14,23,15,31,4,20,8,2,14


In [13]:
RegSeasonSwitch.loc[RegSeasonSwitch['WLoc'] == 'H', 'WLoc'] = 'A'
RegSeasonSwitch.loc[RegSeasonSwitch['WLoc'] == 'A', 'WLoc'] = 'H'
RegSeason.columns.values[6] = 'location'
RegSeasonSwitch.columns.values[6] = 'location'

RegSeason.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(RegSeason.columns)]
RegSeasonSwitch.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(RegSeasonSwitch.columns)]

In [14]:
RegSeason.head(15)

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,...,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
5,2003,11,1458,81,1186,55,H,0,26,57,...,11,12,17,6,22,8,19,4,3,25
6,2003,12,1161,80,1236,62,H,0,23,55,...,15,20,28,9,21,11,30,10,4,28
7,2003,12,1186,75,1457,61,N,0,28,62,...,17,17,23,8,25,10,15,14,8,18
8,2003,12,1194,71,1156,66,N,0,28,58,...,18,12,27,13,26,13,25,8,2,18
9,2003,12,1458,84,1296,56,H,0,32,67,...,14,7,12,9,23,10,18,1,3,18


In [15]:
RegSeasonSwitch.head(15)

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,...,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF
0,2003,10,1328,62,1104,68,N,0,22,53,...,14,11,18,14,24,13,23,7,1,22
1,2003,10,1393,63,1272,70,N,0,24,67,...,20,10,19,15,28,16,13,4,4,18
2,2003,11,1437,61,1266,73,N,0,22,73,...,18,17,29,17,26,15,10,5,2,25
3,2003,11,1457,50,1296,56,N,0,18,49,...,9,17,31,6,19,11,12,14,2,18
4,2003,11,1208,71,1400,77,N,0,24,62,...,14,11,13,17,22,12,14,4,4,20
5,2003,11,1186,55,1458,81,H,0,20,46,...,12,23,27,12,24,12,9,9,3,18
6,2003,12,1236,62,1161,80,H,0,19,41,...,8,32,39,13,18,14,17,11,1,25
7,2003,12,1457,61,1186,75,N,0,20,59,...,14,15,21,13,35,19,19,7,2,21
8,2003,12,1156,66,1194,71,N,0,24,52,...,11,10,18,9,22,9,17,9,2,23
9,2003,12,1296,56,1458,84,H,0,23,52,...,17,15,19,14,22,11,6,12,0,13


Looks like theyre still in order. I thought about just combining them and having 2 rows per game, but im thinking thats probably not a great idea since id be training on games that will also be in the test set, so im going to randomly select 50 percent from RegSeason where team 1 is the willer and the remaining indices will come from RegSeasonSwitch where team2 is the winner


In [16]:
RegSeason.shape

(186662, 34)

In [17]:
RegSeasonSwitch.shape

(186662, 34)

In [18]:
half = RegSeason.shape[0]//2

print(half)
np.random.seed(123)
half_indices = random.sample(list(RegSeason.index), k=half)

RegSeasonHalf = RegSeason.iloc[half_indices].sort_values(['Season', 'DayNum'])
print(RegSeasonHalf.head(20))

RegSeasonSwitchHalf = RegSeasonSwitch.drop(half_indices).sort_values(['Season', 'DayNum'])
RegSeasonSwitchHalf.head(20)

93331
    Season  DayNum  T1_TeamID  T1_Score  T2_TeamID  T2_Score location  NumOT  \
2     2003      11       1266        73       1437        61        N      0   
7     2003      12       1186        75       1457        61        N      0   
10    2003      13       1166       106       1426        50        H      0   
12    2003      13       1237        66       1135        65        N      0   
13    2003      13       1323        76       1125        48        H      0   
21    2003      14       1353        60       1162        36        H      0   
22    2003      14       1390        61       1131        57        H      0   
16    2003      14       1161        81       1194        56        H      0   
26    2003      15       1194        93       1236        86        N      2   
25    2003      15       1161        77       1156        63        H      0   
30    2003      16       1314        71       1353        67        H      0   
32    2003      17       1196     

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,...,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF
0,2003,10,1328,62,1104,68,N,0,22,53,...,14,11,18,14,24,13,23,7,1,22
1,2003,10,1393,63,1272,70,N,0,24,67,...,20,10,19,15,28,16,13,4,4,18
3,2003,11,1457,50,1296,56,N,0,18,49,...,9,17,31,6,19,11,12,14,2,18
4,2003,11,1208,71,1400,77,N,0,24,62,...,14,11,13,17,22,12,14,4,4,20
5,2003,11,1186,55,1458,81,H,0,20,46,...,12,23,27,12,24,12,9,9,3,18
6,2003,12,1236,62,1161,80,H,0,19,41,...,8,32,39,13,18,14,17,11,1,25
8,2003,12,1156,66,1194,71,N,0,24,52,...,11,10,18,9,22,9,17,9,2,23
9,2003,12,1296,56,1458,84,H,0,23,52,...,17,15,19,14,22,11,6,12,0,13
11,2003,13,1106,73,1202,74,N,0,29,63,...,13,9,11,6,21,18,15,7,1,5
14,2003,14,1135,77,1125,83,N,1,28,60,...,31,12,15,15,18,22,16,18,5,21


In [19]:
RegSeasonGames = pd.concat([RegSeasonHalf, RegSeasonSwitchHalf]).sort_index()
RegSeasonGames.head(25)

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,...,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF
0,2003,10,1328,62,1104,68,N,0,22,53,...,14,11,18,14,24,13,23,7,1,22
1,2003,10,1393,63,1272,70,N,0,24,67,...,20,10,19,15,28,16,13,4,4,18
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1457,50,1296,56,N,0,18,49,...,9,17,31,6,19,11,12,14,2,18
4,2003,11,1208,71,1400,77,N,0,24,62,...,14,11,13,17,22,12,14,4,4,20
5,2003,11,1186,55,1458,81,H,0,20,46,...,12,23,27,12,24,12,9,9,3,18
6,2003,12,1236,62,1161,80,H,0,19,41,...,8,32,39,13,18,14,17,11,1,25
7,2003,12,1186,75,1457,61,N,0,28,62,...,17,17,23,8,25,10,15,14,8,18
8,2003,12,1156,66,1194,71,N,0,24,52,...,11,10,18,9,22,9,17,9,2,23
9,2003,12,1296,56,1458,84,H,0,23,52,...,17,15,19,14,22,11,6,12,0,13


In [20]:
RegSeasonGames.shape

(186662, 34)

## COmputing Season Stats By Team

Now to compute season averages. I Want to group by Team 1. I guess i can use a df with every game twice since theyre just averages


In [21]:
RegSeasonDouble = pd.concat([RegSeason, RegSeasonSwitch])
RegSeasonDouble.shape

(373324, 34)

In [22]:
RegSeasonDouble.columns

Index(['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID', 'T2_Score',
       'location', 'NumOT', 'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_FTM',
       'T1_FTA', 'T1_OR', 'T1_DR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_Blk',
       'T1_PF', 'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_FTM', 'T2_FTA',
       'T2_OR', 'T2_DR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF'],
      dtype='object')

In [23]:
RegSeasonDouble.columns = [x.replace('T1_','').replace('T2_','Opp_') for x in list(RegSeasonDouble.columns)]
RegSeasonDouble.columns

Index(['Season', 'DayNum', 'TeamID', 'Score', 'Opp_TeamID', 'Opp_Score',
       'location', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR',
       'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Opp_FGM', 'Opp_FGA', 'Opp_FGM3',
       'Opp_FGA3', 'Opp_FTM', 'Opp_FTA', 'Opp_OR', 'Opp_DR', 'Opp_Ast',
       'Opp_TO', 'Opp_Stl', 'Opp_Blk', 'Opp_PF'],
      dtype='object')

### Look Into Calculating some more advanced stats

Below is just very basic averages

In [24]:
RegSeasonDouble['Games'] = 1

In [25]:
StatsAvgs = RegSeasonDouble.groupby(['TeamID', 'Season'])[['Score', 'Opp_Score',
       'location','Games', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR',
       'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Opp_FGM', 'Opp_FGA', 'Opp_FGM3',
       'Opp_FGA3', 'Opp_FTM', 'Opp_FTA', 'Opp_OR', 'Opp_DR', 'Opp_Ast',
       'Opp_TO', 'Opp_Stl', 'Opp_Blk', 'Opp_PF']].mean().reset_index()

In [45]:
StatsAvgs.columns

Index(['TeamID', 'Season', 'Score', 'Opp_Score', 'Games', 'NumOT', 'FGM',
       'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl',
       'Blk', 'PF', 'Opp_FGM', 'Opp_FGA', 'Opp_FGM3', 'Opp_FGA3', 'Opp_FTM',
       'Opp_FTA', 'Opp_OR', 'Opp_DR', 'Opp_Ast', 'Opp_TO', 'Opp_Stl',
       'Opp_Blk', 'Opp_PF'],
      dtype='object')

## TO DO

Figure out which columns to keep that we want in the season info for each team
Rename columns to specify they are mean values

In [46]:
SA = StatsAvgs[['TeamID', 'Season', 'Score', 'Opp_Score', 'FGM', 'FGM3', 'OR', 'DR', 'Ast', 'TO', 'Stl',
       'Blk', 'PF', 'Opp_FGM', 'Opp_FGM3', 'Opp_OR', 'Opp_DR', 'Opp_Ast', 'Opp_TO', 'Opp_Stl',
       'Opp_Blk', 'Opp_PF']]

In [49]:
SA.columns = ['TeamID', 'Season', 'P_PG', 'PA_PG', 'FGM_PG', 'FGM3_PG', 'OR_PG', 'DR_PG', 'Ast_PG', 'TO_PG', 'Stl_PG',
       'Blk_PG', 'PF_PG', 'Opp_FGM_PG', 'Opp_FGM3_PG', 'Opp_OR_PG', 'Opp_DR_PG', 'Opp_Ast_PG', 'Opp_TO_PG', 'Opp_Stl_PG',
       'Opp_Blk_PG', 'Opp_PF_PG']

In [50]:
SA

,TeamID,Season,P_PG,PA_PG,FGM_PG,FGM3_PG,OR_PG,DR_PG,Ast_PG,TO_PG,...,PF_PG,Opp_FGM_PG,Opp_FGM3_PG,Opp_OR_PG,Opp_DR_PG,Opp_Ast_PG,Opp_TO_PG,Opp_Stl_PG,Opp_Blk_PG,Opp_PF_PG
0,1101,2014,63.142857,78.619048,20.333333,6.666667,8.000000,20.333333,10.000000,15.000000,...,21.571429,27.142857,6.000000,10.380952,24.095238,15.571429,12.142857,7.000000,5.000000,18.666667
1,1101,2015,61.000000,71.857143,21.428571,7.250000,8.250000,19.642857,11.857143,12.821429,...,20.607143,25.035714,6.178571,10.035714,25.892857,12.928571,13.464286,5.857143,4.250000,16.535714
2,1101,2016,69.851852,76.259259,23.814815,6.851852,8.185185,22.518519,12.888889,13.407407,...,22.370370,26.481481,5.592593,8.592593,25.222222,12.851852,13.444444,5.111111,3.296296,19.666667
3,1101,2017,67.880000,72.640000,24.440000,7.080000,7.560000,22.880000,13.600000,14.480000,...,22.160000,24.840000,7.080000,10.640000,25.040000,13.600000,14.800000,6.040000,3.160000,17.240000
4,1101,2018,70.666667,72.814815,25.518519,6.481481,9.037037,23.222222,13.888889,14.407407,...,21.037037,24.777778,6.629630,9.925926,25.037037,12.037037,15.481481,6.518519,2.925926,17.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3476,2023,59.178571,65.428571,21.571429,6.178571,8.678571,22.571429,14.000000,14.535714,...,14.964286,25.178571,5.000000,8.785714,22.964286,11.785714,11.214286,7.892857,3.214286,16.142857
12853,3476,2024,51.391304,69.086957,18.304348,5.478261,9.391304,21.565217,10.826087,19.434783,...,14.304348,27.130435,5.391304,11.434783,24.739130,14.217391,14.043478,11.565217,3.434783,16.608696
12854,3477,2023,64.625000,69.625000,22.937500,3.656250,11.312500,23.875000,11.875000,16.718750,...,15.437500,26.406250,6.531250,9.593750,23.406250,15.437500,15.781250,9.000000,4.062500,18.562500
12855,3477,2024,68.000000,75.600000,24.500000,7.000000,8.450000,26.600000,10.850000,13.000000,...,15.850000,28.000000,5.700000,11.900000,31.000000,13.850000,13.400000,6.650000,3.800000,16.300000


In [27]:
StatsTotals = RegSeasonDouble.groupby(['TeamID', 'Season'])[['Score', 'Opp_Score',
       'location', 'Games', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR',
       'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Opp_FGM', 'Opp_FGA', 'Opp_FGM3',
       'Opp_FGA3', 'Opp_FTM', 'Opp_FTA', 'Opp_OR', 'Opp_DR', 'Opp_Ast',
       'Opp_TO', 'Opp_Stl', 'Opp_Blk', 'Opp_PF']].sum().reset_index()
StatsTotals

,TeamID,Season,Score,Opp_Score,Games,NumOT,FGM,FGA,FGM3,FGA3,...,Opp_FGA3,Opp_FTM,Opp_FTA,Opp_OR,Opp_DR,Opp_Ast,Opp_TO,Opp_Stl,Opp_Blk,Opp_PF
0,1101,2014,1326,1651,21,3,427,1053,140,375,...,340,385,542,218,506,327,255,147,105,392
1,1101,2015,1708,2012,28,0,600,1482,203,537,...,468,437,636,281,725,362,377,164,119,463
2,1101,2016,1886,2059,27,2,643,1456,185,509,...,409,478,674,232,681,347,363,138,89,531
3,1101,2017,1697,1816,25,4,611,1333,177,477,...,490,397,595,266,626,340,370,151,79,431
4,1101,2018,1908,1966,27,1,689,1525,175,539,...,533,449,633,268,676,325,418,176,79,477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3476,2023,1657,1832,28,1,604,1563,173,567,...,443,282,405,246,643,330,314,221,90,452
12853,3476,2024,1182,1589,23,0,421,1278,126,464,...,416,217,330,263,569,327,323,266,79,382
12854,3477,2023,2068,2228,32,1,734,1863,117,456,...,633,329,468,307,749,494,505,288,130,594
12855,3477,2024,1360,1512,20,1,490,1327,140,432,...,389,278,385,238,620,277,268,133,76,326


In [28]:
StatsTotals['FG_PCT'] = StatsTotals['FGM']/StatsTotals['FGA']
StatsTotals['FG_PCT3'] = StatsTotals['FGM3']/StatsTotals['FGA3']
StatsTotals['FT_PCT'] = StatsTotals['FTM']/StatsTotals['FTA']

StatsTotals['Opp_FG_PCT'] = StatsTotals['Opp_FGM']/StatsTotals['Opp_FGA']
StatsTotals['Opp_FG_PCT3'] = StatsTotals['Opp_FGM3']/StatsTotals['Opp_FGA3']
StatsTotals['Opp_FT_PCT'] = StatsTotals['Opp_FTM']/StatsTotals['Opp_FTA']

#A possesion can only end with a shot, turnover, foul (free trhrows) i think. We consider an offensive rebound a continuation of that possesion
StatsTotals['POSS'] = StatsTotals['FGA']+StatsTotals['TO']+(StatsTotals['FTA']*0.44)-StatsTotals['OR']
StatsTotals['POSS_PER_GAME'] = StatsTotals['POSS']/StatsTotals['Games']

# THeoretically opponent POSS should only differer by 1 or 2 at most, but since Im calculating it in kind of a janky way, I want to see how much difference there is.
StatsTotals['Opp_POSS'] = StatsTotals['Opp_FGA']+StatsTotals['Opp_TO']+(StatsTotals['Opp_FTA']*0.44)-StatsTotals['Opp_OR']
StatsTotals['Opp_POSS_PER_GAME'] = StatsTotals['Opp_POSS']/StatsTotals['Games']

StatsTotals['POSS_Diff'] = (StatsTotals['POSS'] -StatsTotals['Opp_POSS'])/StatsTotals['Games'] 
 
StatsTotals.sort_values('POSS_Diff').loc[StatsTotals.Season == 2024].tail(25)

,TeamID,Season,Score,Opp_Score,Games,NumOT,FGM,FGA,FGM3,FGA3,...,FG_PCT3,FT_PCT,Opp_FG_PCT,Opp_FG_PCT3,Opp_FT_PCT,POSS,POSS_PER_GAME,Opp_POSS,Opp_POSS_PER_GAME,POSS_Diff
12435,3436,2024,1428,1285,24,0,565,1294,111,388,...,0.286082,0.708333,0.386328,0.281899,0.692063,1515.16,63.131667,1483.60,61.816667,1.315000
12849,3474,2024,1221,1514,21,1,447,1284,86,327,...,0.262997,0.658470,0.427336,0.311594,0.703901,1585.04,75.478095,1557.16,74.150476,1.327619
7607,1473,2024,1383,1615,22,1,467,1206,100,360,...,0.277778,0.736287,0.460184,0.368889,0.717791,1514.56,68.843636,1485.16,67.507273,1.336364
2757,1235,2024,1886,1488,24,0,674,1415,160,451,...,0.354767,0.694853,0.402755,0.338710,0.714286,1672.36,69.681667,1639.88,68.328333,1.353333
7867,3119,2024,1159,1300,20,0,425,1159,93,344,...,0.270349,0.701299,0.454294,0.319865,0.719870,1381.52,69.076000,1354.08,67.704000,1.372000
10,1101,2024,1549,1663,22,3,533,1300,114,348,...,0.327586,0.726378,0.466238,0.337531,0.705545,1622.52,73.750909,1592.12,72.369091,1.381818
11626,3381,2024,1395,1587,21,1,535,1266,129,436,...,0.295872,0.636364,0.447568,0.348946,0.742857,1529.52,72.834286,1500.00,71.428571,1.405714
7687,3105,2024,1344,1415,22,2,504,1321,82,304,...,0.269737,0.699725,0.410025,0.335616,0.733333,1598.72,72.669091,1567.40,71.245455,1.423636
6556,1415,2024,1534,1649,23,0,545,1423,193,609,...,0.316913,0.693370,0.458952,0.354610,0.720621,1576.28,68.533913,1542.44,67.062609,1.471304
710,1138,2024,1530,1838,23,1,567,1386,158,596,...,0.265101,0.687861,0.483803,0.362348,0.721519,1645.24,71.532174,1610.80,70.034783,1.497391


In [29]:
#DReb_PCT
StatsTotals['DREB_PCT'] = StatsTotals['DR']/(StatsTotals['DR']+StatsTotals['Opp_OR'])
#OReb_PCT
StatsTotals['OREB_PCT'] = StatsTotals['OR']/(StatsTotals['OR']+StatsTotals['Opp_DR'])
#EFG_PCT
StatsTotals['EFG_PCT'] = (StatsTotals['FGM']+StatsTotals['FGM3']*0.5)/StatsTotals['FGA']
#Opp_EFG_PCT
StatsTotals['Opp_EFG_PCT'] = (StatsTotals['Opp_FGM']+StatsTotals['Opp_FGM3']*0.5)/StatsTotals['Opp_FGA']
#OFF_RTG
StatsTotals['OFF_RTG'] = 100*(StatsTotals['Score']/StatsTotals['POSS'])
#DEF_RTG
StatsTotals['DEF_RTG'] = 100*(StatsTotals['Opp_Score']/StatsTotals['Opp_POSS'])
#NET_RTG
StatsTotals['NET_RTG'] = StatsTotals['OFF_RTG']/StatsTotals['DEF_RTG']
#TOV_PCT
StatsTotals['TOV_PCT'] = StatsTotals['TO']/StatsTotals['POSS']

In [51]:
StatsTotals

,TeamID,Season,Score,Opp_Score,Games,NumOT,FGM,FGA,FGM3,FGA3,...,Opp_POSS_PER_GAME,POSS_Diff,DREB_PCT,OREB_PCT,EFG_PCT,Opp_EFG_PCT,OFF_RTG,DEF_RTG,NET_RTG,TOV_PCT
0,1101,2014,1326,1651,21,3,427,1053,140,375,...,66.546667,-0.080000,0.662016,0.249258,0.471985,0.564171,94.999284,118.141226,0.804116,0.225677
1,1101,2015,1708,2012,28,0,600,1482,203,537,...,64.280000,-0.195714,0.661853,0.241632,0.473347,0.553020,95.187142,111.787714,0.851499,0.200071
2,1101,2016,1886,2059,27,2,643,1456,185,509,...,69.502222,-0.788148,0.723810,0.245011,0.505151,0.545549,101.655815,109.722045,0.926485,0.195119
3,1101,2017,1697,1816,25,4,611,1333,177,477,...,68.312000,0.094400,0.682578,0.231902,0.524756,0.528689,99.230481,106.335636,0.933182,0.211676
4,1101,2018,1908,1966,27,1,689,1525,175,539,...,70.167407,-0.102222,0.700559,0.265217,0.509180,0.517394,100.858460,103.772987,0.971914,0.205629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3476,2023,1657,1832,28,1,604,1563,173,567,...,67.221429,0.444286,0.719818,0.274266,0.441779,0.473716,87.457248,97.332908,0.898537,0.214817
12853,3476,2024,1182,1589,23,0,421,1278,126,464,...,70.747826,0.542609,0.653491,0.275159,0.378717,0.482419,72.087237,97.652409,0.738202,0.272614
12854,3477,2023,2068,2228,32,1,734,1863,117,456,...,74.122500,-0.680000,0.713352,0.325833,0.425389,0.482470,87.994009,93.932342,0.936781,0.227644
12855,3477,2024,1360,1512,20,1,490,1327,140,432,...,78.920000,-0.804000,0.690909,0.214195,0.422005,0.447426,87.050028,95.793208,0.908729,0.166419


In [30]:
StatsTotals.columns

Index(['TeamID', 'Season', 'Score', 'Opp_Score', 'Games', 'NumOT', 'FGM',
       'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl',
       'Blk', 'PF', 'Opp_FGM', 'Opp_FGA', 'Opp_FGM3', 'Opp_FGA3', 'Opp_FTM',
       'Opp_FTA', 'Opp_OR', 'Opp_DR', 'Opp_Ast', 'Opp_TO', 'Opp_Stl',
       'Opp_Blk', 'Opp_PF', 'FG_PCT', 'FG_PCT3', 'FT_PCT', 'Opp_FG_PCT',
       'Opp_FG_PCT3', 'Opp_FT_PCT', 'POSS', 'POSS_PER_GAME', 'Opp_POSS',
       'Opp_POSS_PER_GAME', 'POSS_Diff', 'DREB_PCT', 'OREB_PCT', 'EFG_PCT',
       'Opp_EFG_PCT', 'OFF_RTG', 'DEF_RTG', 'NET_RTG', 'TOV_PCT'],
      dtype='object')

## TO Do

Determine which columns to keep to be merged later into the season info dataset
Rename columns if necessary

In [52]:
ST = StatsTotals[['TeamID', 'Season', 'FG_PCT', 'FG_PCT3', 'FT_PCT', 'Opp_FG_PCT',
       'Opp_FG_PCT3', 'Opp_FT_PCT', 'POSS_PER_GAME',
       'Opp_POSS_PER_GAME', 'DREB_PCT', 'OREB_PCT', 'EFG_PCT',
       'Opp_EFG_PCT', 'OFF_RTG', 'DEF_RTG', 'NET_RTG', 'TOV_PCT']]

Games Data Doesnt need the stats anymore

In [31]:
RegSeasonGames.columns

Index(['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID', 'T2_Score',
       'location', 'NumOT', 'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_FTM',
       'T1_FTA', 'T1_OR', 'T1_DR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_Blk',
       'T1_PF', 'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_FTM', 'T2_FTA',
       'T2_OR', 'T2_DR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF'],
      dtype='object')

In [32]:
RegSeasonGame = RegSeasonGames[['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID', 'T2_Score',
       'location']]
RegSeasonGame

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location
0,2003,10,1328,62,1104,68,N
1,2003,10,1393,63,1272,70,N
2,2003,11,1266,73,1437,61,N
3,2003,11,1457,50,1296,56,N
4,2003,11,1208,71,1400,77,N
...,...,...,...,...,...,...,...
186657,2024,100,3427,70,3409,74,H
186658,2024,100,3424,67,3161,64,H
186659,2024,100,3296,47,3444,65,H
186660,2024,100,3457,54,3347,51,H


Eventually ill Have to merge the RegSeasonGame with that teams season stats, I want to make some more advanced stats frist though
When I merge, Remember column names will ahve to be the same whcih will probably involve making 2 identical datasets out of the Stats Df, only differnce being the column names where 'TeamId' will become 'T1_TeamID' in one and 'T2_TeamID' in the other. All other columns will change similarly. For instance, 'Opp_FGM" becomes 'T1_Opp_FGM'

Also Ill have to do the same for tourney games and conference tourney games

I can already merge GameRecords and Stats

In [33]:
TeamRecords.shape

(12857, 7)

In [34]:
#Stats.shape

In [35]:
temp = pd.concat([WConf, MConf], ignore_index = True)
TeamRundown = pd.merge(TeamRecords, temp, 'left', on = ['Season', 'TeamID']).rename(columns = {'ConfAbbrev':'Conference'})
TeamRundown

,TeamID,Season,Wins,Losses,OpponentsBeaten,OpponentsLost,Win_PCT,Conference
0,1101,2014,2.0,19.0,"[1146, 1223]","[1182, 1382, 1268, 1234, 1395, 1462, 1406, 130...",0.095238,southland
1,1101,2015,7.0,21.0,"[1170, 1354, 1115, 1146, 1311, 1322, 1223]","[1409, 1182, 1415, 1222, 1260, 1129, 1213, 137...",0.250000,southland
2,1101,2016,9.0,18.0,"[1197, 1146, 1322, 1270, 1249, 1223, 1249, 136...","[1339, 1161, 1324, 1306, 1304, 1415, 1395, 117...",0.333333,southland
3,1101,2017,9.0,16.0,"[1306, 1149, 1372, 1249, 1309, 1358, 1322, 127...","[1149, 1328, 1307, 1170, 1362, 1309, 1368, 114...",0.360000,southland
4,1101,2018,12.0,15.0,"[1144, 1132, 1102, 1402, 1358, 1270, 1223, 132...","[1117, 1252, 1367, 1252, 1403, 1309, 1368, 137...",0.444444,southland
...,...,...,...,...,...,...,...,...
12852,3421,2013,0.0,28.0,[],"[3441, 3459, 3184, 3314, 3378, 3144, 3367, 324...",0.000000,big_south
12853,3438,2021,0.0,5.0,[],"[3416, 3187, 3241, 3155, 3199]",0.000000,acc
12854,3451,2013,0.0,29.0,[],"[3140, 3129, 3329, 3119, 3428, 3307, 3430, 346...",0.000000,big_sky
12855,3464,2010,0.0,30.0,[],"[3189, 3219, 3245, 3145, 3419, 3451, 3338, 310...",0.000000,horizon


In [36]:
TeamRundown['Opp_wins'] = 0
TeamRundown['Opp_losses'] = 0
count = 0
for row in range(len(TeamRundown)):
    for opp in TeamRundown['OpponentsBeaten'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'Opp_wins'] += TeamRundown.loc[mask, 'Wins'].sum()
        TeamRundown.at[row, 'Opp_losses'] += TeamRundown.loc[mask, 'Losses'].sum()
    for opp in TeamRundown['OpponentsLost'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'Opp_wins'] += TeamRundown.loc[mask, 'Wins'].sum()
        TeamRundown.at[row, 'Opp_losses'] += TeamRundown.loc[mask, 'Losses'].sum()
    if row <= 4:
        print(TeamRundown['Opp_wins'][row])
        print(TeamRundown['Opp_losses'][row])
        

288
311
379
418
338
445
322
393
353
418


In [37]:
TeamRundown['Opp_win_pct'] = TeamRundown['Opp_wins']/(TeamRundown['Opp_wins']+TeamRundown['Opp_losses'])
TeamRundown    

,TeamID,Season,Wins,Losses,OpponentsBeaten,OpponentsLost,Win_PCT,Conference,Opp_wins,Opp_losses,Opp_win_pct
0,1101,2014,2.0,19.0,"[1146, 1223]","[1182, 1382, 1268, 1234, 1395, 1462, 1406, 130...",0.095238,southland,288,311,0.480801
1,1101,2015,7.0,21.0,"[1170, 1354, 1115, 1146, 1311, 1322, 1223]","[1409, 1182, 1415, 1222, 1260, 1129, 1213, 137...",0.250000,southland,379,418,0.475533
2,1101,2016,9.0,18.0,"[1197, 1146, 1322, 1270, 1249, 1223, 1249, 136...","[1339, 1161, 1324, 1306, 1304, 1415, 1395, 117...",0.333333,southland,338,445,0.431673
3,1101,2017,9.0,16.0,"[1306, 1149, 1372, 1249, 1309, 1358, 1322, 127...","[1149, 1328, 1307, 1170, 1362, 1309, 1368, 114...",0.360000,southland,322,393,0.450350
4,1101,2018,12.0,15.0,"[1144, 1132, 1102, 1402, 1358, 1270, 1223, 132...","[1117, 1252, 1367, 1252, 1403, 1309, 1368, 137...",0.444444,southland,353,418,0.457847
...,...,...,...,...,...,...,...,...,...,...,...
12852,3421,2013,0.0,28.0,[],"[3441, 3459, 3184, 3314, 3378, 3144, 3367, 324...",0.000000,big_south,416,396,0.512315
12853,3438,2021,0.0,5.0,[],"[3416, 3187, 3241, 3155, 3199]",0.000000,acc,59,49,0.546296
12854,3451,2013,0.0,29.0,[],"[3140, 3129, 3329, 3119, 3428, 3307, 3430, 346...",0.000000,big_sky,461,390,0.541716
12855,3464,2010,0.0,30.0,[],"[3189, 3219, 3245, 3145, 3419, 3451, 3338, 310...",0.000000,horizon,452,450,0.501109


In [38]:
TeamRundown['Opp_Opp_wins'] = 0
TeamRundown['Opp_Opp_losses'] = 0
count = 0
for row in range(len(TeamRundown)):
    for opp in TeamRundown['OpponentsBeaten'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'Opp_Opp_wins'] += TeamRundown.loc[mask, 'Opp_wins'].sum()
        TeamRundown.at[row, 'Opp_Opp_losses'] += TeamRundown.loc[mask, 'Opp_losses'].sum()
    for opp in TeamRundown['OpponentsLost'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'Opp_Opp_wins'] += TeamRundown.loc[mask, 'Opp_wins'].sum()
        TeamRundown.at[row, 'Opp_Opp_losses'] += TeamRundown.loc[mask, 'Opp_losses'].sum()
    if row <= 4:
        print(TeamRundown['Opp_Opp_wins'][row])
        print(TeamRundown['Opp_Opp_losses'][row])

8783
9022
10911
12334
10840
12292
9944
10933
10502
12069


In [39]:
TeamRundown['Opp_Opp_win_pct'] = TeamRundown['Opp_Opp_wins']/(TeamRundown['Opp_Opp_wins']+TeamRundown['Opp_Opp_losses'])
TeamRundown   

,TeamID,Season,Wins,Losses,OpponentsBeaten,OpponentsLost,Win_PCT,Conference,Opp_wins,Opp_losses,Opp_win_pct,Opp_Opp_wins,Opp_Opp_losses,Opp_Opp_win_pct
0,1101,2014,2.0,19.0,"[1146, 1223]","[1182, 1382, 1268, 1234, 1395, 1462, 1406, 130...",0.095238,southland,288,311,0.480801,8783,9022,0.493288
1,1101,2015,7.0,21.0,"[1170, 1354, 1115, 1146, 1311, 1322, 1223]","[1409, 1182, 1415, 1222, 1260, 1129, 1213, 137...",0.250000,southland,379,418,0.475533,10911,12334,0.469391
2,1101,2016,9.0,18.0,"[1197, 1146, 1322, 1270, 1249, 1223, 1249, 136...","[1339, 1161, 1324, 1306, 1304, 1415, 1395, 117...",0.333333,southland,338,445,0.431673,10840,12292,0.468615
3,1101,2017,9.0,16.0,"[1306, 1149, 1372, 1249, 1309, 1358, 1322, 127...","[1149, 1328, 1307, 1170, 1362, 1309, 1368, 114...",0.360000,southland,322,393,0.450350,9944,10933,0.476314
4,1101,2018,12.0,15.0,"[1144, 1132, 1102, 1402, 1358, 1270, 1223, 132...","[1117, 1252, 1367, 1252, 1403, 1309, 1368, 137...",0.444444,southland,353,418,0.457847,10502,12069,0.465287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3421,2013,0.0,28.0,[],"[3441, 3459, 3184, 3314, 3378, 3144, 3367, 324...",0.000000,big_south,416,396,0.512315,11053,12826,0.462875
12853,3438,2021,0.0,5.0,[],"[3416, 3187, 3241, 3155, 3199]",0.000000,acc,59,49,0.546296,1227,999,0.551213
12854,3451,2013,0.0,29.0,[],"[3140, 3129, 3329, 3119, 3428, 3307, 3430, 346...",0.000000,big_sky,461,390,0.541716,12234,12946,0.485862
12855,3464,2010,0.0,30.0,[],"[3189, 3219, 3245, 3145, 3419, 3451, 3338, 310...",0.000000,horizon,452,450,0.501109,12958,13997,0.480727


In [40]:
TeamRundown['SOS'] = (2*TeamRundown['Opp_win_pct']+TeamRundown['Opp_Opp_win_pct'])/3
TeamRundown['SOS'].describe()

count    12857.000000
mean         0.503947
std          0.043072
min          0.369173
25%          0.472160
50%          0.498573
75%          0.535500
max          0.637620
Name: SOS, dtype: float64

In [41]:
TeamRundown.columns

Index(['TeamID', 'Season', 'Wins', 'Losses', 'OpponentsBeaten',
       'OpponentsLost', 'Win_PCT', 'Conference', 'Opp_wins', 'Opp_losses',
       'Opp_win_pct', 'Opp_Opp_wins', 'Opp_Opp_losses', 'Opp_Opp_win_pct',
       'SOS'],
      dtype='object')

In [44]:
TR = TeamRundown[['TeamID', 'Season', 'Wins', 'Losses', 'Conference','SOS']]

In [53]:
print(TR.shape, SA.shape, ST.shape)

(12857, 6) (12857, 22) (12857, 18)


### Time to merge Our 3 dfs containg team season stats

In [54]:
temp1 = pd.merge(TR, SA, 'left', on = ['TeamID', 'Season'])

In [55]:
Teams = pd.merge(temp1, ST, 'left', on = ['TeamID', 'Season'])

In [57]:
Teams.shape


(12857, 42)

In [58]:
Teams.to_csv('Data\\TeamInfo.csv', index=False)